In [8]:
# Function for turbidity processing
# 1.Return data columns within the corresponding time period
import pandas as pd
# 1.数据处理，返回值: 查询对应时间内得到的数据列
def data_pro(file_name, lie_name, begin_time, end_time):
    # 使用绝对路径加载时出错
    # 筛选特定时间数据
    data_1107 = pd.read_excel("./mongdb_data/20231206/data/"+file_name)
    # 筛选特定时间，方法1
    # 思路：将时间作为一个文本数据，使用contain()方法进行筛选
    # 把时间设置为索引
    data_1107 = data_1107.set_index("currentTime")
    # 将索引转为日期格式
    # 若报错，则 https://blog.csdn.net/qq_39467576/article/details/88864398
    data_1107.index = pd.to_datetime(data_1107.index)
    # 筛选时间
    data_lie = data_1107.between_time(begin_time, end_time).loc[: , lie_name]
    data_var = data_1107.between_time(begin_time, end_time).loc[: , lie_name][0:].var()
    data_mean = data_1107.between_time(begin_time, end_time).loc[: , lie_name].mean()
    # return data_1107, data_var, data_mean
    return data_lie

# 2.Return outlier data index
import numpy as np
from sklearn.covariance import EllipticEnvelope
import copy
from scipy.interpolate import lagrange
from sklearn.impute import KNNImputer # knn填充
# 删除前建议先对副本进行操作
# 2.四分位法，找到异常值的下标
def indicies_of_outliers(x):
    # percentile函数返回区域中数值的第 k 个百分点的值。
    # 同时传入两个参数表示25%和75%
    q1, q3 = np.percentile(x, [25, 75])
    iqr = q3 - q1
    lower_bound = q1 - (iqr * 1.5)
    upper_bound = q3 + (iqr * 1.5)
    return np.where((x > upper_bound) | (x < lower_bound))

# 拉格朗日插值法填充
def lagrange_value (data, k=2):     # 输入需处理的数据（空值用nan填充），k是空值前后取得数据个数
    # 取每一列
    col_nan = data[data.isnull()]   # 取每一列缺失值
    col_nan_list = col_nan.index    # 取每一列缺失值的index
    for n in col_nan_list:          # 对每一列缺失值index进行遍历，定位需要插值的位置
        list1 = list(range(n-k,n))  # 缺失值前取k个值
        if list1[0] < data.index[0]:# 判断缺失值前是否有k个值，没有的话就有多少取多少
            list1 = list(range(data.index[0],n)) 
        list2 = data[n:]            # 缺失值后取k个非空值
        list2 = list2[list2.notnull()]
        list2 = list(list2[0:k].index)
        y=data[list1+list2]         # 取数
        data[n] = lagrange(y.index,list(y))(n)  # 填充缺失值
    return data

# 3.This is an outlier filling function
def data_pro_outer(filename, lie_name, begin_time, end_time, method):
    data_lie = \
    data_pro(filename, lie_name, begin_time, end_time)
    data_var, data_mean = data_lie.var(), data_lie.mean()
    feature = data_lie

    # 2023年4月7日
    var_list = []
    # 执行函数,返回下标如下所示
    index = indicies_of_outliers(feature)
    index = list(index)
    type(index)
    value = feature[index[0]]
    copy_data_lie = copy.deepcopy(data_lie)
    type(data_lie)
    # data_lie[index[0][2]]
    # 异常数据比例
    rate = len(index[0]) / len(data_lie)
    original_mean, original_var = copy_data_lie.mean(), copy_data_lie.var()
    # 尝试使用下标删除数据
    len(copy_data_lie.index[index[0]]), len(index[0])
    copy_data_lie.index[index[0]]# 找到需要删除的下标
    len1 = len(copy_data_lie)
    # print("原始均值:",copy_data_lie.mean(), "原始方差:", copy_data_lie.var()) 
    # 调用drop函数,注意是否选择就地删除,
    # 我们应该把inplace开关打开，因为它默认是关闭的。即不会影响原来的元素
    var_list.append(copy_data_lie.mean())
    if method == "mode": # 众数方法
        print("len1:",len(index[0]), "len2:",len(copy_data_lie), "异常数据比例：", rate)
        print("众数填充")
        #index[0]获取下标
        # copy_data_lie[copy_data_lie.index[index[0]]] = copy_data_lie.mode()
        # 2023年7月4日，copy_data_lie.index[index[0]]索引获取异常数据中出错，尝试修改如下所示。
        # 2023年7月4日，pandas统计值分析 https://blog.csdn.net/sinat_23971513/article/details/114911104
        # print("mode:", copy_data_lie.mode())
        # print("float:", float(copy_data_lie.mode()))
        # mode()	众数 （返回一个dataframe格式的数据），以下这句报错
        # copy_data_lie[index[0]] = copy_data_lie.mode()
        # 尝试将其转为浮点型,如果有多个众数，则取第一个，否则会报错
        print("copy_data_lie.mode():", copy_data_lie.mode()[0])
        copy_data_lie[index[0]] = float(copy_data_lie.mode()[0])
    elif method == "ave": # average
        print("平均数填充")
        # copy_data_lie[copy_data_lie.index[index[0]]] = copy_data_lie.mean()
        # 2023年7月4日，copy_data_lie.index[index[0]]索引获取异常数据中出错，尝试修改如下所示。
        copy_data_lie[index[0]] = copy_data_lie.mean()
    elif method == "median":
        print("中位数填充")
        # copy_data_lie[copy_data_lie.index[index[0]]] = copy_data_lie.median()
        # 2023年7月4日，copy_data_lie.index[index[0]]索引获取异常数据中出错，尝试修改如下所示。
        copy_data_lie[index[0]] = copy_data_lie.median()
    elif method == "chazhi":
        print("插值填充")
        # 注意再调用reset_index()时，会返回一个新的值，对copy_data_lie本身是没有影响的
        # drop=True表示把之前的删除
        # print("copy_data_lie.index[index[0]]:", copy_data_lie.index[index[0]])
        # 2023年7月4日，copy_data_lie.index[index[0]]索引获取异常数据中出错，尝试修改如下所示。
        # copy_data_lie[copy_data_lie.index[index[0]]] = np.nan
        copy_data_lie[index[0]] = np.nan
        new_data_lie = copy_data_lie.reset_index(drop=True)
        result = lagrange_value(new_data_lie, k=1)
        # 把经过插值处理的结果返回给copy_data_lie便于打印
        copy_data_lie = result
    elif method == "kmean":
        print("knn填充")
        # 先将异常值设置为nan
        # print("copy_data_lie.index[index[0]]:", copy_data_lie.index[index[0]])
        # copy_data_lie[copy_data_lie.index[index[0]]] = np.nan
        # 2023年7月4日
        copy_data_lie[index[0]] = np.nan
        # copy_data_lie.to_csv('./copy_data_nan.csv')
        np.savetxt('./copy_data_nan.csv', copy_data_lie)
        new_data_lie = copy_data_lie.reset_index(drop=True)
        impter = KNNImputer(n_neighbors=3, weights='distance')
        new_data_lie = impter.fit_transform(np.array(new_data_lie).reshape(-1, 1))
        copy_data_lie = new_data_lie
        np.savetxt('./copy_data_lie.csv', copy_data_lie)
        # copy_data_lie.to_csv('./copy_data_nan.csv')
    elif method == "bfill":
        print("使用前一个数据填充")
        # 使用前一个数据填充
        # 先将异常值设置为nan
        # copy_data_lie[copy_data_lie.index[index[0]]] = np.nan
        # 2023年7月4日
        copy_data_lie[index[0]] = np.nan
        copy_data_lie.fillna(method='bfill', inplace=True)
    else:
        print("删除")
        # print("len1:", len(copy_data_lie))
        copy_data_lie.drop(copy_data_lie.index[index[0]], inplace=True)
        # print("len2:", len(copy_data_lie))
        # copy_data_lie.drop(copy_data_lie.index[index[0]], inplace=True)
    
    print("均值:",copy_data_lie.mean(), "方差:", copy_data_lie.var(), "\n")
    # id(copy_data_lie), id(data_lie)
    # print(copy_data_lie)
    len2 = len(copy_data_lie)
    # 用于数据展示len1表示原来的长度，len2表示处理异常值后的长度
    # 只要不使用drop方法，len1和len2就是一样的
    len1, len2, len1-len2,len(index[0])
    if len(index[0]) == 0:
        return -1
    outlier_data = data_lie[index[0]]
    return outlier_data, rate, original_mean, original_var

# len1, len2, sub_len,outliers_len, data_lie = \
    # data_pro_outer("data_tds_174_ec_345_1209.xlsx", "ec", "21:51:50", "22:18:00", "bfill")
# len1, len2, sub_len,outliers_len


'''
浊度：5份水样的的5个完整监测时间段
不同的文件名列表如下所示
数据1：device_2022_11_21_2.xlsx
对应时间："20:31:28", "20:50:00"
数据2：data_device1_985.56_1209_2.xlsx
对应时间："20:31:28", "20:53:00"
数据3：data_tds_174_ec_345_1209_2.xlsx 
对应时间："21:49:50", "22:23:00"
数据4：device1_221107_2.xlsx
对应时间："15:26:00", "15:28:00"
数据5：device_20221114_2.xlsx
对应时间："21:31:28", "21:52:00"
data6: data_221013.xlsx
对应时间："22:34:00", "22:50:00"
'''
# method_list = ["kmean", "ave", "mode", "median", "bfill", "drop", "chazhi"]
method_list = ["mode", "ave", "median", "chazhi", "kmean", "bfill", "drop"]

for i in method_list:
    outlier_data,rate, original_mean, original_var = \
    data_pro_outer("data_221013.xlsx", 
    "zhuodu", "22:34:00", "22:50:00", i)
print("异常数据比例：", rate ,"原始均值:",original_mean.mean(), "原始方差:", original_var) 
# print(outlier_data)
# 找到异常值后
# ①将其删除
# 异常值对应的下标为index[0]
# copy_data_lie.mode() 方差: 95.5058753609828
outlier_data # 在下一单元格打印不重复的值

len1: 24 len2: 775 异常数据比例： 0.03096774193548387
众数填充
copy_data_lie.mode(): 109.78
均值: 100.61141935483872 方差: 34.08386103192464 

平均数填充
均值: 100.31210514047866 方差: 31.39520786797084 

中位数填充
均值: 100.31629677419356 方差: 31.394100867883637 

插值填充
均值: 100.44154193548388 方差: 32.88174988080646 

knn填充
均值: 100.31841544607191 方差: 31.353451889523647 

使用前一个数据填充
均值: 100.40651554404144 方差: 32.38191248613937 

删除
均值: 100.30813333333334 方差: 32.36231960836663 

异常数据比例： 0.03096774193548387 原始均值: 100.11464516129033 原始方差: 58.847178394598714


currentTime
2023-12-19 22:34:54     79.83
2023-12-19 22:34:55     78.82
2023-12-19 22:34:56     72.85
2023-12-19 22:34:57    139.56
2023-12-19 22:34:59     79.53
2023-12-19 22:35:00     76.47
2023-12-19 22:35:01     77.95
2023-12-19 22:35:02    136.46
2023-12-19 22:35:03    143.51
2023-12-19 22:35:04     63.29
2023-12-19 22:46:09     72.90
2023-12-19 22:46:11    132.26
2023-12-19 22:46:11    134.70
2023-12-19 22:48:24     77.95
2023-12-19 22:48:25    143.51
2023-12-19 22:48:26     72.72
2023-12-19 22:48:28     63.29
2023-12-19 22:49:48     79.55
2023-12-19 22:49:49     78.01
2023-12-19 22:49:50     72.72
2023-12-19 22:49:51    138.70
2023-12-19 22:49:58     78.48
2023-12-19 22:49:59     78.48
2023-12-19 22:50:00     78.18
Name: zhuodu, dtype: float64

In [9]:
from collections import OrderedDict

original_list = outlier_data.values
unique_list = list(OrderedDict.fromkeys(original_list))

for i in unique_list:
    print("%.2f"%i)

79.83
78.82
72.85
139.56
79.53
76.47
77.95
136.46
143.51
63.29
72.90
132.26
134.70
72.72
79.55
78.01
138.70
78.48
78.18
